In [ ]:
# Import the TensorFlow library, which provides a wide range of tools for machine learning and deep learning tasks.
import tensorflow as tf

# Import specific modules from Keras, which is a high-level neural networks API 
# integrated within TensorFlow for building and training deep learning models.
from tensorflow.keras import layers, models

# Import the ImageDataGenerator class from Keras, which is used for real-time data augmentation 
# and loading images from directories, helping to preprocess images for training and validation.
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# Define the paths for training and validation data
train_data_dir = 'data/train'  # Directory for training images
validation_data_dir = 'data/validation'  # Directory for validation images

In [ ]:
# Create a list of dictionaries to store label information for hand gestures used in the sign language detection model.
# Each dictionary contains a 'name' representing the gesture and an 'id' that serves as a unique identifier for that gesture.
label_info = [
    {'name': 'hello', 'id': 1},       # 'hello' gesture mapped to ID 1
    {'name': 'thanks', 'id': 2},      # 'thanks' gesture mapped to ID 2
    {'name': 'yes', 'id': 3},         # 'yes' gesture mapped to ID 3
    {'name': 'no', 'id': 4},          # 'no' gesture mapped to ID 4
    {'name': 'iloveyou', 'id': 5}     # 'I love you' gesture mapped to ID 5
]


In [ ]:
# Create a dictionary that maps class indices to corresponding label names.
# This is done using a dictionary comprehension that enumerates over the label_info list.
label_dict = {i: label['name'] for i, label in enumerate(label_info)}

# Print the resulting label dictionary to verify the mapping.
print("Label dictionary:", label_dict)  # Output the label dictionary to the console.


In [ ]:
# Define the height and width of the images to be processed.
# These dimensions should be adjusted based on the requirements of your model 
# and the characteristics of your dataset. All images will be resized to this size.
img_height, img_width = 150, 150  # Image dimensions (height, width)

# Set the batch size for processing images during training and validation.
# The batch size determines how many images will be fed into the model at once 
# for training or evaluation, impacting the speed and memory usage during training.
batch_size = 4  # Number of images to process in a batch


In [ ]:
# Create an ImageDataGenerator for the training dataset to apply real-time data augmentation.
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,               # Rescale pixel values to the range [0, 1] for normalization.
    rotation_range=20,               # Randomly rotate images in the range of 0 to 20 degrees.
    width_shift_range=0.2,           # Randomly shift images horizontally by up to 20% of the image width.
    height_shift_range=0.2,          # Randomly shift images vertically by up to 20% of the image height.
    shear_range=0.2,                 # Apply shear transformations randomly to the images.
    zoom_range=0.2,                  # Randomly zoom in on images by up to 20%.
    horizontal_flip=True,             # Randomly flip images horizontally.
    fill_mode='nearest'               # Fill in new pixels created during transformations with the nearest pixel values.
)

# Create an ImageDataGenerator for the validation dataset.
# For validation, only rescaling is performed to maintain the integrity of the validation dataset.
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Normalize validation images to the range [0, 1]


In [ ]:
# Load training data using the flow_from_directory method of the ImageDataGenerator.
# This method generates batches of tensor image data from the training data directory.
train_generator = train_datagen.flow_from_directory(
    train_data_dir,                     # Directory containing the training data organized into subdirectories for each class.
    target_size=(img_height, img_width),  # Resize all images to the specified target size (img_height, img_width).
    batch_size=batch_size,               # Number of images to yield per batch.
    class_mode='categorical'             # Specifies the type of label arrays that are returned: 'categorical' means one-hot encoded labels.
)

# Load validation data using the flow_from_directory method of the ImageDataGenerator.
# Similar to the training data generator, it generates batches of tensor image data from the validation data directory.
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,                # Directory containing the validation data organized into subdirectories for each class.
    target_size=(img_height, img_width),  # Resize all images to the specified target size (img_height, img_width).
    batch_size=batch_size,               # Number of images to yield per batch.
    class_mode='categorical'             # Specifies the type of label arrays that are returned: 'categorical' means one-hot encoded labels.
)


In [ ]:
# Build the CNN model
# Input layer: Specifies the shape of the input images
input_layer = layers.Input(shape=(img_height, img_width, 3))

# First Convolutional Layer: Applies 32 filters of size 3x3 and uses ReLU activation
x = layers.Conv2D(32, (3, 3), activation='relu')(input_layer)

# First Max Pooling Layer: Reduces the spatial dimensions of the feature maps by a factor of 2
x = layers.MaxPooling2D((2, 2))(x)

# Batch Normalization: Normalizes the output of the previous layer to improve training speed and stability
x = layers.BatchNormalization()(x)

# Second Convolutional Layer: Applies 64 filters of size 3x3 and uses ReLU activation
x = layers.Conv2D(64, (3, 3), activation='relu')(x)

# Second Max Pooling Layer: Again reduces the spatial dimensions of the feature maps
x = layers.MaxPooling2D((2, 2))(x)

# Batch Normalization: Normalizes the output of the previous layer
x = layers.BatchNormalization()(x)

# Third Convolutional Layer: Applies 128 filters of size 3x3 and uses ReLU activation
x = layers.Conv2D(128, (3, 3), activation='relu')(x)

# Third Max Pooling Layer: Reduces the spatial dimensions of the feature maps
x = layers.MaxPooling2D((2, 2))(x)

# Batch Normalization: Normalizes the output of the previous layer
x = layers.BatchNormalization()(x)

# Flatten Layer: Flattens the 3D output of the previous layer into a 1D vector
x = layers.Flatten()(x)

# Fully Connected Layer: A dense layer with 128 neurons and ReLU activation
x = layers.Dense(128, activation='relu')(x)

# Dropout Layer: Randomly sets 50% of the input units to 0 to prevent overfitting during training
x = layers.Dropout(0.5)(x) 

# Output Layer: A dense layer with a number of units equal to the number of classes in the dataset
# It uses softmax activation to output class probabilities
output_layer = layers.Dense(train_generator.num_classes, activation='softmax')(x)

In [ ]:
# Create the model
model = models.Model(inputs=input_layer, outputs=output_layer)  # Instantiate the model with the specified input and output layers.


In [ ]:
# Compile the model
model.compile(optimizer='adam',  # Use the Adam optimizer for training, which is efficient and widely used.
              loss='categorical_crossentropy',  # Specify categorical crossentropy as the loss function for multi-class classification.
              metrics=['accuracy'])  # Track accuracy as the performance metric during training and evaluation.


In [ ]:
# Model summary
model.summary()

In [ ]:
# Custom callback to stop training at 92% accuracy
class StopAtAccuracy(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):  # This method is called at the end of each epoch during training.
        if logs["accuracy"] >= 0.92:  # Check if the accuracy for the epoch has reached or exceeded 92%.
            self.model.stop_training = True  # Stop the training process if the condition is met.


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Learning Rate Reduction Callback: This callback reduces the learning rate when a metric has stopped improving.
# It monitors the validation loss ('val_loss') during training.
# If the validation loss does not improve for a specified number of epochs (patience), 
# the learning rate will be reduced by a factor (factor).
# The 'verbose' parameter, when set to 1, will print a message when the learning rate is reduced.

lr_reduction = ReduceLROnPlateau(
    monitor='val_loss',  # The metric to monitor for improvement
    factor=0.2,          # Factor by which the learning rate will be reduced (new_lr = lr * factor)
    patience=3,          # Number of epochs with no improvement after which learning rate will be reduced
    verbose=1            # If set to 1, prints a message when the learning rate is reduced
)


In [ ]:
epochs = 100

# Train the model using the fit method, which adjusts the model's weights based on training data
history = model.fit(
    train_generator,  # The training data generator providing batches of training images and labels
    validation_data=validation_generator,  # The validation data generator for evaluating the model during training
    epochs=epochs,  # The number of complete passes through the training dataset
    callbacks=[StopAtAccuracy(), lr_reduction]  # Include callbacks for stopping the training early and reducing the learning rate
)


In [ ]:
import matplotlib.pyplot as plt  # Import the matplotlib library for plotting.

# Create a figure with a specified size for the plots.
plt.figure(figsize=(12, 4))

# Create the first subplot for accuracy values.
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
plt.plot(history.history['accuracy'], label='Train Accuracy')  # Plot training accuracy.
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')  # Plot validation accuracy.
plt.title('Model Accuracy')  # Title of the accuracy plot.
plt.ylabel('Accuracy')  # Label for the y-axis.
plt.xlabel('Epoch')  # Label for the x-axis.
plt.legend(loc='upper left')  # Add a legend to the plot.

# Create the second subplot for loss values.
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
plt.plot(history.history['loss'], label='Train Loss')  # Plot training loss.
plt.plot(history.history['val_loss'], label='Validation Loss')  # Plot validation loss.
plt.title('Model Loss')  # Title of the loss plot.
plt.ylabel('Loss')  # Label for the y-axis.
plt.xlabel('Epoch')  # Label for the x-axis.
plt.legend(loc='upper left')  # Add a legend to the plot.

plt.tight_layout()  # Adjust layout to prevent overlap of subplots.
plt.show()  # Display the plots.


In [ ]:
# Save the model after training in the Keras format
model.save('sign_language_detection_model.keras')  # Save the entire model architecture, weights, and training configuration to a file named 'sign_language_detection_model.keras'.
